In [267]:
import os

from python_speech_features import mfcc
from python_speech_features import delta
from python_speech_features import logfbank
import scipy.io.wavfile as wav
from random import randint
# import librosa

# import tensorflow as tf

In [468]:
# every frame is considered independent

no_of_features = 13
no_of_fbank_features = 13
no_of_columns = (3 * no_of_features) + no_of_fbank_features
test_frames = 40
classes = 6

def get_feature_vectors(dataset_type):
    
    #set parameters for training and testing
    if (dataset_type == "train"):
        directory = os.path.join(os.getcwd(), 'data_thuyg20_sre/enroll')
        no_of_frames = 400
        start_frame = 10
    elif (dataset_type == "test"):    
        directory = os.path.join(os.getcwd(), 'data_thuyg20_sre/test')
        no_of_frames = test_frames
        start_frame = randint(1, 200)
        
    dataset = numpy.empty([0, no_of_columns + 1])
    
    for file in os.listdir(directory):
        
        # filter speakers
        n = int(classes/2)
        names = []
        
        for i in range (1, n):
            names.append('F1' + '%02d'%(i))
            names.append('M1' + '%02d'%(i))

        if any(name in file for name in names):
            
            # extract mfcc vectors
            (rate,sig) = wav.read(os.path.join(directory, file))
            fbank_feat = logfbank(sig,rate)
            mfcc_feat = mfcc(sig,rate,winlen=0.032,winstep=0.016,numcep=13,nfft=2048)
#             print("Rate :" + str(rate) + " " + file)
#             print("Signal :" + str(sig) + " " + file)
#             y, sr = librosa.load(os.path.join(directory, file))
#             mfcc_feat = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13, n_fft=2048).T
            d_mfcc_feat = delta(mfcc_feat, 2)
            dd_mfcc_feat = delta(d_mfcc_feat, 2)
            
#             fbank_feat = logfbank(sig,rate)
            mfcc_vectors = mfcc_feat[start_frame:start_frame+no_of_frames,:no_of_features]
            dmfcc_vectors = d_mfcc_feat[start_frame:start_frame+no_of_frames,:no_of_features]
            ddmfcc_vectors = dd_mfcc_feat[start_frame:start_frame+no_of_frames,:no_of_features]
            fbank_vectors = fbank_feat[start_frame:start_frame+no_of_frames,:no_of_fbank_features]
            
            feature_vectors = numpy.hstack((mfcc_vectors, dmfcc_vectors, ddmfcc_vectors, fbank_vectors))
#             print(feature_vectors.shape)
            
            # get speaker index from filename
            speaker_index = file.split("_")[0]
            if speaker_index[0] == 'M':
                speaker_index = int(classes/2) + int(speaker_index[3:])
            else:
                speaker_index = int(speaker_index[3:])

            #append speaker index to feature vectors
            np_speaker_index = numpy.array([speaker_index])
            temp = numpy.tile(np_speaker_index[numpy.newaxis,:], (feature_vectors.shape[0],1))
            concatenated_feature_vector = numpy.concatenate((feature_vectors,temp), axis=1)
            
#             print(concatenated_feature_vector.shape)
#             print(fbank_vectors.shape)
            
            # append file's data to dataset
            dataset = numpy.concatenate((dataset, concatenated_feature_vector), axis=0)
            

    return dataset


In [469]:
from keras.models import Sequential
import numpy as numpy 

In [470]:
# from numpy import genfromtxt
my_data = get_feature_vectors("train")


In [471]:
# numpy.set_printoptions(threshold=numpy.nan)
# print(my_data)
print(my_data.shape)

(1600, 53)


In [472]:
Y = numpy.copy(my_data[:, no_of_columns:])
print(Y.shape)

(1600, 1)


In [473]:
X = numpy.copy(my_data[:, :no_of_columns])
print(X.shape)
mean = X.mean(0, keepdims=True)

print(mean.shape)
std_deviation = numpy.std(X, axis=0, keepdims=True)
print(std_deviation.shape)

normalized_X = (X - mean) / std_deviation
print(normalized_X.shape)


(1600, 52)
(1, 52)
(1, 52)
(1600, 52)


In [474]:
from keras import utils as np_utils
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD

one_hot_labels = np_utils.to_categorical(Y, num_classes=classes)
print(one_hot_labels)

[[ 0.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  1.  0.]
 ..., 
 [ 0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  1.]]


In [475]:
def train():

    # MultiLayer Perceptron
    model = Sequential()

    # 3000 0.6 1500 0.6 -> 57% acc
    # 2560 0.7 1280 0.7 -> 60% acc
    # 2560 0.7 1600 0.7 -> 51% acc
    # 2560 0.6 1600 0.7 -> 49%
    # 2560 0.7 1280 -   -> 50
    # 3000 0.7 1280 0.7 -> 54% acc
    # 2560 0.7 1280 0.8 -> 45% acc

    model.add(Dense(3000, activation='tanh', input_dim=no_of_columns))
    model.add(Dense(100, activation='tanh'))
    model.add(Dropout(0.3))
    model.add(Dense(100, activation='tanh'))
    model.add(Dropout(0.3))
    model.add(Dense(100, activation='tanh'))
    model.add(Dropout(0.3))
    model.add(Dense(100, activation='tanh'))
    model.add(Dropout(0.3))
    
    model.add(Dense(classes, activation='softmax'))

    sgd = SGD(lr=0.005, decay=1e-5, momentum=0.9, nesterov=True)
    model.compile(loss='categorical_crossentropy',
                  optimizer=sgd,
                  metrics=['accuracy'])

    model.fit(normalized_X, one_hot_labels, epochs=10, batch_size=32)
    
    return model
# score = model.evaluate(x_test, y_test, batch_size=128)

In [476]:
def test(model):
    
    test_model = get_feature_vectors("test")
    print(test_model.shape)

    test_X = numpy.copy(test_model[:, :no_of_columns])
#     print(test_X.shape)

    normalized_test_X = (test_X - mean) / std_deviation
#     print(normalized_test_X.shape)

    test_Y = numpy.copy(test_model[:, no_of_columns:])
#     print(test_Y.shape)
    test_labels = np_utils.to_categorical(test_Y, num_classes=classes)

#     print(model.test_on_batch(test_X, test_labels, sample_weight=None))
#     print(model.metrics_names)
    predictions = model.predict(test_X)
    
    b = [sum(predictions[current: current+test_frames]) for current in range(0, len(predictions), test_frames)]
    predicted_Y = []
    for row in b:
        predicted_Y.append(row.argmax(axis=0))
    
    for t, p in zip(test_Y[::test_frames].T[0], predicted_Y):
        print (int(t), p)
    
    diff = predicted_Y - test_Y[::test_frames].T[0]
    correct = sum(x == 0 for x in diff)
    total = len(predicted_Y)
    percent = correct/total * 100
    print(str(correct) + " / " + str(total) + " = " + str(percent))
    
    loss = str(numpy.sum(numpy.abs(diff)))
    print("Loss = " + loss)
    
    return correct, percent, loss

In [343]:
model = train()
test(model)

Epoch 1/10
4000/4000 [==============================] - 3s 767us/step - loss: 1.1619 - acc: 0.5895
Epoch 2/10
4000/4000 [==============================] - 3s 812us/step - loss: 0.6201 - acc: 0.7678
Epoch 3/10
4000/4000 [==============================] - 4s 915us/step - loss: 0.5362 - acc: 0.8028
Epoch 4/10
4000/4000 [==============================] - 3s 867us/step - loss: 0.4791 - acc: 0.8245
Epoch 5/10
4000/4000 [==============================] - 3s 836us/step - loss: 0.4643 - acc: 0.8293
Epoch 6/10
4000/4000 [==============================] - 3s 868us/step - loss: 0.4098 - acc: 0.8473
Epoch 7/10
4000/4000 [==============================] - 3s 785us/step - loss: 0.3931 - acc: 0.8588
Epoch 8/10
4000/4000 [==============================] - 3s 802us/step - loss: 0.3535 - acc: 0.8728
Epoch 9/10
4000/4000 [==============================] - 4s 900us/step - loss: 0.3517 - acc: 0.8768
Epoch 10/10
4000/4000 [==============================] - 4s 903us/step - loss: 0.3472 - acc: 0.8787


/home/rohan/anaconda3/lib/python3.6/site-packages/scipy/io/wavfile.py:273: WavFileWarning: Chunk (non-data) not understood, skipping it.
  WavFileWarning)


(2280, 53)
6 6
2 1
4 4
9 7
2 1
3 4
7 9
3 7
1 8
1 1
7 7
6 6
9 9
1 8
1 8
3 4
4 4
1 1
9 9
4 4
4 4
2 1
6 6
2 1
9 9
7 4
3 3
7 7
2 1
6 6
3 4
6 6
4 4
8 8
7 7
7 9
3 4
7 7
4 4
1 1
9 9
8 8
3 4
6 6
1 8
4 4
9 9
7 3
9 9
1 1
2 1
3 4
6 6
6 6
9 9
8 8
8 8
35 / 57 = 61.4035087719
Loss = 57.0


(35, 61.403508771929829, '57.0')

In [480]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution1D, MaxPooling1D
from keras.utils import np_utils

from matplotlib import pyplot as plt

def cnn_train():
    
    temp = normalized_X.reshape(normalized_X.shape[0], no_of_columns, 1)
    
    model = Sequential()
    # 13 7 1 1 0.25 60 0.25 10 - 70%
    model.add(Convolution1D(52, 13, activation='tanh', input_shape=(no_of_columns,1)))
    model.add(Convolution1D(52, 7, activation='tanh', use_bias=True))
    model.add(Convolution1D(13, 3, activation='tanh', use_bias=True))
    # stride = 2 - 70
    # 20, 10, 17 op - 64
    
    model.add(MaxPooling1D(pool_size=(1)))
#     model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(1000, activation='tanh'))
    model.add(Dropout(0.25))
    # 0.4 70
    model.add(Dense(classes, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    model.fit(temp, one_hot_labels, epochs=classes if classes > 10 else 10, batch_size=32)
    return model

In [478]:
def test_cnn(model):
    
    test_model = get_feature_vectors("test")
#     print(test_model.shape)

    test_X = numpy.copy(test_model[:, :no_of_columns])
#     print(test_X.shape)

    normalized_test_X = (test_X - mean) / std_deviation
#     print(normalized_test_X.shape)

    test_Y = numpy.copy(test_model[:, no_of_columns:])
#     print(test_Y.shape)
    test_labels = np_utils.to_categorical(test_Y, num_classes=classes)
    
    test_X = test_X.reshape(test_X.shape[0], no_of_columns, 1)
    normalized_test_X = normalized_test_X.reshape(normalized_test_X.shape[0], no_of_columns, 1)
    
    print(model.test_on_batch(test_X, test_labels, sample_weight=None))
    print(model.metrics_names)
    predictions = model.predict(test_X)

    b = [sum(predictions[current: current+test_frames]) for current in range(0, len(predictions), test_frames)]
    predicted_Y = []
    for row in b:
        predicted_Y.append(row.argmax(axis=0))

    # print(predicted_Y)
    # print(test_Y[::40].T)
    
    indices = numpy.argmax(predictions, axis=1)
    majority = []
    
    for i in range(0, len(indices), test_frames):
        majority.append(find_majority(indices[i:i + test_frames]))
        
#     majority = 
#     for t, p, m in zip(test_Y[::test_frames].T[0], predicted_Y, majority):
#         print(int(t), p, m[0])
    
#     for t, p in zip(test_Y.T[0], indices):
#         print(int(t), p)
    
    
    diff = predicted_Y - test_Y[::test_frames].T[0]
    maj_diff = numpy.array(majority)[:, 0] - test_Y[::test_frames].T[0]

    numerator = sum(x == 0 for x in diff)
    denominator = len(predicted_Y)
    
    numerator2 = sum(x == 0 for x in maj_diff)
    denominator2 = len(maj_diff)
    

    print("Accuracy prob_diff: {} of {} - {}".format(numerator, denominator, numerator/denominator))
    
    print("Accuracy maj_diff: {} of {} - {}".format(numerator2, denominator2, numerator2/denominator2))
    

In [481]:
cnn_model = cnn_train()
test_cnn(cnn_model)

Epoch 1/10
1600/1600 [==============================] - 3s 2ms/step - loss: 0.9242 - acc: 0.6344
Epoch 2/10
1600/1600 [==============================] - 5s 3ms/step - loss: 0.4758 - acc: 0.8187
Epoch 3/10
1600/1600 [==============================] - 6s 4ms/step - loss: 0.3551 - acc: 0.8694
Epoch 4/10
1600/1600 [==============================] - 4s 3ms/step - loss: 0.3241 - acc: 0.8775
Epoch 5/10
1600/1600 [==============================] - 3s 2ms/step - loss: 0.2607 - acc: 0.8919
Epoch 6/10
1600/1600 [==============================] - 3s 2ms/step - loss: 0.2196 - acc: 0.9188
Epoch 7/10
1600/1600 [==============================] - 3s 2ms/step - loss: 0.1590 - acc: 0.9300
Epoch 8/10
1600/1600 [==============================] - 4s 2ms/step - loss: 0.1231 - acc: 0.9506
Epoch 9/10
1600/1600 [==============================] - 4s 3ms/step - loss: 0.1044 - acc: 0.9556
Epoch 10/10
1600/1600 [==============================] - 4s 2ms/step - loss: 0.0834 - acc: 0.9738A: 1s - lo
[array(2.9235022068

In [466]:
# test(model)
test_cnn(cnn_model)

[array(5.629917621612549, dtype=float32), array(0.2573298513889313, dtype=float32)]
['loss', 'acc']
Accuracy prob_diff: 77 of 191 - 0.4031413612565445
Accuracy maj_diff: 74 of 191 - 0.387434554973822


In [30]:

import python_speech_features
python_speech_features.__file__

# print(predicted_Y)
# print(test_Y[::40].T)

# for t, p in zip(test_Y[::40].T[0], predicted_Y):
#     print (int(t), p)



'/home/rohan/anaconda3/lib/python3.6/site-packages/python_speech_features-0.6-py3.6.egg/python_speech_features/__init__.py'

In [ ]:
# majority

# argmax_pred = numpy.argmax(predictions, axis=1)
# argmax_pred = argmax_pred.reshape((-1, 25))

# from scipy.stats import mode
# argmax_pred = mode(argmax_pred, axis=-1)[0]

# # print(argmax_pred)
# # print(test_Y)

# for t, p in zip(test_Y[::40].T[0], argmax_pred):
#     print (int(t), p)

In [283]:
def find_majority(k):
    myMap = {}
    maximum = ( '', 0 ) # (occurring element, occurrences)
    for n in k:
        if n in myMap: myMap[n] += 1
        else: myMap[n] = 1

        # Keep track of maximum on the go
        if myMap[n] > maximum[1]: maximum = (n,myMap[n])

    return maximum

NameError: name 'predictions' is not defined